In [1]:
import os
import pandas as pd
import plotly.express as px

In [2]:
rootwd=os.getcwd()
Svc_ID_Info_path=os.path.join(rootwd, "data", "Svc_ID_Info.csv")
Svc_Std_Result_path=os.path.join(rootwd, "data", "Svc_Std_Result.csv")

Svc_ID_Info=pd.read_csv(Svc_ID_Info_path)
Svc_Std_Result=pd.read_csv(Svc_Std_Result_path)

In [3]:
departments=Svc_ID_Info["department_name_en"].unique()
list(sorted(departments, key=lambda x: x[0])) #visually checking data uniqueness and integrity
len(departments)

81

In [4]:
Svc22_23 = Svc_ID_Info[Svc_ID_Info["fiscal_yr"]=="2022-2023"]
len(Svc22_23["service_id"].unique())
#Svc_ID_Info.iloc[:,0:15]

1637

In [5]:
Year_Depart=Svc_ID_Info[["fiscal_yr", "department_name_en"]]

df={}
for year in Year_Depart["fiscal_yr"].unique():
    df[year]=Year_Depart[Year_Depart["fiscal_yr"]==year]["department_name_en"].unique()

for key, val in df.items():
    print(f"{key}: {len(val)}")

2018-2019: 72
2022-2023: 74
2019-2020: 76
2020-2021: 77
2021-2022: 78


In [6]:
res=Svc22_23[["service_id","total_applications"]]
res=res[res["total_applications"].notna()].sort_values(by="total_applications")
display(res.head(10))
display(res.tail(10))

,service_id,total_applications
7179,SRV03550,0.0
7178,SRV03549,0.0
7177,SRV03548,0.0
16,1004,0.0
21,1005,0.0
26,1006,0.0
7160,SRV03531,0.0
7161,SRV03532,0.0
7167,SRV03538,0.0
7169,SRV03540,0.0


,service_id,total_applications
1465,1422,3064022.0
1,1001,3286418.0
765,1242,3429561.0
4295,2238,3464403.0
6849,SRV03049,3706829.0
5088,37,10078890.0
2720,1726,10104531.0
5382,728,22976516.0
2523,1677,57000000.0
5237,669,70361874.0


In [7]:
df=Svc22_23[["e_registration","e_authentication","e_application","e_decision","e_issuance","e_feedback"]].fillna("Napplicable")

res={}
for key, val in df.items():
    NYes=sum(val=="Y")
    NNo=sum(val=="N")
    Napp=sum(val=="Napplicable")
    
    res[key]={"Online": NYes, "NotOnline":NNo, "NotApplicable":Napp}
    
res=pd.DataFrame(res).transpose().reset_index()
res=res.melt(id_vars="index", value_vars=["Online",	"NotOnline", "NotApplicable"], var_name="Status", value_name="Value")
res["Percentage"]=round(res["Value"]*100/1637,1)

fig=px.bar(res, x='index',y="Value", color='Status',
            text="Value")

# Update layout
fig.update_layout(
    xaxis_title='ServiceType',
    yaxis_title='Number of Services'
)

# Show plot
fig.show()
fig.write_image(os.path.join(rootwd, "TexFiles", "Figures","OnlineStatus.png"), width=1000, scale=3)



In [8]:
filtered_df = df[(df == 'Y').all(axis=1)]
len(filtered_df)

135

In [9]:
df=Svc_ID_Info[["fiscal_yr","telephone_applications","online_applications","in_person_applications","postal_mail_applications",
                "email_applications","fax_applications", "other_applications"]].fillna(0)
apptype=["telephone", "online", "in_person", "postal_mail", "email", "fax", "other"]
df.columns=["fiscal_yr"]+apptype

res={}
for year in df["fiscal_yr"].unique():    
    res[year]=df[df["fiscal_yr"]==year][apptype]

newdf=pd.DataFrame()
for key, val in res.items():
    temp=val.sum().reset_index()
    temp.columns=["Type", "Value"]
    pcttotal=sum(temp["Value"])
    temp["Percentage"]=round(temp["Value"]*100/pcttotal,1)
    temp["Year"]=[key]*len(apptype)
    temp["labels"]=[f"{x}%" for x in temp["Percentage"]]

    newdf=pd.concat([newdf, temp])

newdf=newdf.sort_values(by="Year")


fig=px.bar(newdf, x='Year',y="Percentage", barmode="group", color='Type',
            text="labels", width=1800,height=600)

# Update layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Percentage',
    legend=dict(font=dict(size=16), 
                orientation="h",  # Horizontal orientation
                yanchor="bottom", # Anchor legend to the bottom
                y=-0.2,           # Place the legend below the plot
                xanchor="center", # Center the legend horizontally
                x=0.5),
    title_x=0.5
)

fig.update_traces(textposition='outside')

# Show plot
fig.show()
fig.write_image(os.path.join(rootwd, "TexFiles", "Figures","TypeApp.png"), scale=3)




In [10]:
df=Svc_Std_Result[["fiscal_yr", "service_id", "target_met"]].fillna("Napplicable")
df

,fiscal_yr,service_id,target_met
0,2022-2023,1043,N
1,2020-2021,1043,Y
2,2021-2022,1043,Y
3,2020-2021,1044,Y
4,2022-2023,890,Y
...,...,...,...
10385,2021-2022,SRV02852,Napplicable
10386,2021-2022,SRV02853,Napplicable
10387,2021-2022,SRV02856,Napplicable
10388,2021-2022,SRV02855,Napplicable


In [11]:
res={}
for year in df["fiscal_yr"].unique():    
    cond1=df["fiscal_yr"]==year
    cond2=df["target_met"]=="Y"
    res[year]=[len(df[df["fiscal_yr"]==year]["service_id"].unique()), len(df[cond1 & cond2])]
res=pd.DataFrame(res).transpose().reset_index()
Typenames=["Services with Standards", "Service Standard Target Met"]
res.columns=["Year"]+Typenames
res=res.sort_values(by="Year")
res

,Year,Services with Standards,Service Standard Target Met
3,2018-2019,723,876
4,2019-2020,791,995
1,2020-2021,888,1301
2,2021-2022,963,1257
0,2022-2023,928,1348


In [12]:
res=res.melt(id_vars="Year", value_vars=Typenames, value_name="Value", var_name="Type")
res["Percentage"] = res.apply(lambda val: round(val["Value"]*100/1637,1) if val["Type"] == Typenames[0] else round(val["Value"]*100/2280,1), axis=1)
res["labels"]=[f"{x}: {y}%" for x, y in zip(res["Value"], res["Percentage"])]

fig=px.bar(res, x='Year',y="Percentage", barmode="group", color='Type', text="labels", width=1800,height=600)

# Update layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Percentage',
    legend=dict(font=dict(size=16), 
                orientation="h",  # Horizontal orientation
                yanchor="bottom", # Anchor legend to the bottom
                y=-0.2,           # Place the legend below the plot
                xanchor="center", # Center the legend horizontally
                x=0.5),
    title_x=0.5
)

fig.update_traces(textposition='outside')
# Show plot
fig.show()

fig.write_image(os.path.join(rootwd, "TexFiles", "Figures","StdWithTarget.png"), scale=3)
